# 环境安装

In [ ]:
!pip install ray
!pip install cython
!pip install line_profiler

# 代码实现

In [2]:
import numpy as np
import pandas as pd

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [3]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [ ]:
%%timeit
target_mean_v1(data,'y','x')

In [ ]:
from cProfile import Profile
prof = Profile()
prof.enable()
target_mean_v1(data,'y','x')
prof.create_stats()
prof.print_stats()

In [ ]:
from line_profiler import LineProfiler
profile = LineProfiler(target_mean_v1)
profile.enable()
target_mean_v1(data, 'y', 'x')
profile.disable()
profile.print_stats()

In [4]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [ ]:
%%timeit
target_mean_v2(data,'y','x')

In [ ]:
from line_profiler import LineProfiler
profile = LineProfiler(target_mean_v2)
profile.enable()
target_mean_v2(data, 'y', 'x')
profile.disable()
profile.print_stats()

In [6]:
%load_ext Cython

In [16]:
%%cython
# cython: profile=True

import numpy as np
import pandas as pd
cimport numpy as np

def target_mean_v3(data, y_name, x_name):

    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    x_data = data.loc[:,x_name]
    y_data = data.loc[:,y_name]
    for i in range(data.shape[0]):
        if x_data[i] not in value_dict.keys():
            value_dict[x_data[i]] = y_data[i]
            count_dict[x_data[i]] = 1
        else:
            value_dict[x_data[i]] += y_data[i]
            count_dict[x_data[i]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[x_data[i]] - y_data[i]) / (count_dict[x_data[i]] - 1)
    return result

In [19]:
%%timeit
target_mean_v3(data,'y','x')

10 loops, best of 3: 124 ms per loop


In [ ]:
from cProfile import Profile
prof = Profile()
prof.enable()
target_mean_v3(data,'y','x')
prof.create_stats()
prof.print_stats()

In [21]:
%prun -l 7 -s cumulative target_mean_v3(data, 'y', 'x')

# 性能分析器




## cProfile



* ncalls：函数被调用的次数。如果这一列有两个值，就表示有递归调用，第二个值是原生调用次数，第一个值是总调用次数。
* tottime：函数内部消耗的总时间。
* percall：是tottime除以ncalls，一个函数每次调用平均消耗时间。
* cumtime：之前所有子函数消费时间的累计和。
* filename:lineno(function)：被分析函数所在文件名、行号、函数名。



In [ ]:
from cProfile import Profile
prof = Profile()
prof.enable()
target_mean_v3(data,'y','x')
prof.create_stats()
prof.print_stats()

## line_profile
* Line：文件中的行号。
* Hits：性能分析时代码执行的次数。
* Time：一段代码执行的总时间，由计数器决定。
* Per Hit：执行一段代码平均消耗时间。
* % Time：执行一段代码时间消耗比例。

In [ ]:
from line_profiler import LineProfiler
profile = LineProfiler(target_mean_v3)  #把函数传递到性能分析器
profile.enable()  #开始分析
target_mean_v3(data, 'y', 'x')
profile.disable()  #停止分析
profile.print_stats()  #打印出性能分析结果

# 参考内容
* todo